In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import binom
plt.style.use(['seaborn-whitegrid'])
plt.rcParams['figure.figsize'] = (9,6)
plt.rcParams['font.family'] = 'DejaVu Sans'

In [ ]:
n = 35 #кол-во признаков
N = 2  #чиcло классов

s = np.zeros([n,2])#исходные изображения
s[:,0] = l[0]
s[:,1] = l[1]

pw1 = pw[0]
pw2 = pw[1]
pw = [pw1,pw2] #априорные вероятности 

P = 20 #кол-во шагов изменения параметра pi
K = 1000 #кол-во испытаний

ns = sum(abs(s[:,0]-s[:,1])) #несовпадающие элементы
s_ = 1 - s 

pi,p12th,p21th,p12ex,p21ex = np.zeros(P),np.zeros(P),np.zeros(P),np.zeros(P),np.zeros(P)
#pi - вероятность искажения элемента
Pc_ = np.zeros([2,2,P])

for i in range(P):
    pi[i] = (1/P)*i
    pI = pi[i]
    
    if pI == 0: 
        pI=0.0001
    if pI == 0.5: 
        pI=0.4999
        
    pI_ = 1-pI
    G1 = np.zeros(n)
    G2 = np.zeros(n) 
    
    for a in range(n):
        G1[a]=np.log((s[a,0]*pI_+s_[a,0]*pI)/(s[a,1]*pI_+s_[a,1]*pI))
        G2[a]=np.log((s[a,0]*pI+s_[a,0]*pI_)/(s[a,1]*pI+s_[a,1]*pI_))
    l0_ = np.log(pw[1]/pw[0])
    L0 = l0_/(2*np.log(pI_)-2*np.log(pI)) + ns/2
    L0r = np.floor(L0)
    if pI<0.5:
        p12th[i] = binom.cdf(L0r,ns,pI_)
        p21th[i] = 1-binom.cdf(L0r,ns,pI)
    else:
        p12th[i] = 1-binom.cdf(L0r,ns,pI_)
        p21th[i] = binom.cdf(L0r,ns,pI)
    
    for k in range(K):
        for j in range(N):
            x = s[:,j].copy()
            r = np.random.rand(n)
            ir = np.where(r<pI)[0]
            x[ir] = 1 - x[ir]
            x_ = 1 - x
            u = np.dot(G1,x) + np.dot(G2,x_) - l0_
            if u > 0:
                iai = 0
            else:
                iai = 1
            Pc_[iai,j,i] = Pc_[iai,j,i]+1
    Pc_/= K 
    p12ex[i] = Pc_[1,0,i] 
    p21ex[i] = Pc_[0,1,i]
    
plt.rcParams['figure.figsize'] = (15, 12)
ms= 1
axes = plt.gca()
axes.set_xlim(min(pi), max(pi))
axes.set_ylim(-0.01, ms)
plt.plot(pi,p12th,'b',label = 'p12th',linewidth=2.0)
plt.plot(pi,p21th,'r',label = 'p21th',linewidth=2.0)
plt.plot(pi,p12ex,'--oy',label = 'p12ex',linewidth=2.0)
plt.plot(pi,p21ex,'--^g',label = 'p21ex',linewidth=2.0)
plt.title('Теоретические вероятности ошибок и их оценки')
plt.xlabel('pi')
plt.ylabel('P')
ann = 'pw = {}'.format(pw)
plt.annotate(ann,(0.1,0.75*ms)) 
plt.legend();

In [1]:
%store -d l,pw